# Chapter 7 - Combining Different Models for Ensemble Learning
1. 다수결 투표를 기반으로 하는 예측
2. 임의의 훈련 데이터 조합의 반복으로 Overfit 줄이기
3. weak learners로 부터 강한모델 생성하기

## 1 앙상블 학습
여러분류기를 활용해 최적값을 추출

Decision Tree, SVM, Logistic Regression 를 활용해서 학습

개별 분류기의 레이블을 결합 후, 가장 많은 표를 얻은 y^ 레이블을 선택

<img src="https://databricks.com/wp-content/uploads/2015/01/Ensemble-example.png" align='left'> <img src="https://hackpad-attachments.imgix.net/hackpad.com_Gal7qtO4fUl_p.550971_1453750253605_undefined" width='350' align = 'left'>